In [105]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import h5py
import matplotlib.pyplot as plt
'''import inflect
c2h.stringify = inflect.engine()'''
from tqdm import tqdm
from collections import Counter
import csv_to_hdf5 as c2h
import obspy
import math
import creep_event_picker as cep

In [ ]:
directory = '../../Data/DATA_tidied/HDF5/'  # Replace with your actual directory path
files = c2h.list_files_in_directory(directory)
print(files)
del files[10]
files.sort()
del files[1],files[25],files[25],files[25],files[25],files[25],files[25],files[25],files[29],files[37],files[42],files[45],files[45],files[53],files[53],files[54]
print(files)

In [200]:
frequency_dict = {
        1/60: '1S',    # 1 second
        16: '10S',  # 10 seconds
        1/2: '30S',  # 30 seconds
        1: '1T',  # 1 minute
        2: '2T',  # 2 minutes
        5: '5T',  # 5 minutes
        10: '10T',# 10 minutes
        30: '30T',# 30 minutes
        60: '60T'    # 60 minutes
    }

In [201]:
def interpolate(time,slip, freq):
    creeping_df = pd.DataFrame({'Time':time,'Tm':time,'Slip':slip})
    creeping_df.Time = creeping_df.Time.dt.round(freq) #round creep times to nearest mins (make evenly spaced)
    creeping_df.Tm = creeping_df.Tm.dt.round(freq)
    creeping_df.set_index('Time',inplace=True) #set index of the dataframe
    creeping_df.drop_duplicates(subset=['Tm'], inplace=True) 
    upsampled = creeping_df.resample(freq).ffill(1)
    return upsampled

In [202]:
Creepmeter_dataframe_SAC = pd.DataFrame()


In [ ]:
for i in range(len(files)):
    abbreviation = files[i][:4]
    if abbreviation[2]=='.':
        abbreviation = abbreviation[:2]
    print(abbreviation,i)
    # Open the HDF5 file in read mode
    with h5py.File('../../Data/DATA_tidied/HDF5/{k}.h5'.format(k=abbreviation), 'r') as f:
        try:
            author = f.attrs['author']
        except KeyError:
            print('no author')
        network = f.attrs['network']
        latitude = f.attrs['latitude']
        longitude = f.attrs['longitude']
        depth = f.attrs['depth']
        length = f.attrs['length'] 
        obliquity = f.attrs['obliquity']
        # Loop through each key in the file
        for key in f.keys():
            print("Key:", key)
            data = f[key]  # Access the dataset
            try:
                data_keys = list(data.keys())

                slip = data[data_keys[0]][:]
                time =  data[data_keys[1]][:]
                decoded_time = [byte_str.decode('utf-8') for byte_str in time]
                decoded_time = pd.to_datetime(decoded_time)

                smpl_rate = data.attrs['sampling_rate']   
                time_units = data.attrs['time_units']     
                slip_units = data.attrs['slip_units']  
                channel = data.attrs['channel'] 
                starttime = data.attrs['starttime'] 
                delta = data.attrs['delta']
                freq_chosen = frequency_dict[smpl_rate]
                upsampled = interpolate(decoded_time,slip,freq_chosen)


                tr = obspy.Trace(np.array(upsampled['Slip']))
                st = obspy.Stream(tr)
                st[0].stats.network = network
                st[0].stats.station = abbreviation
                st[0].stats.location = '{k} mins'.format(k=smpl_rate)
                st[0].stats.channel = channel
                st[0].stats.starttime = starttime
                st[0].stats.delta = delta
                st[0].stats['latitude'] = latitude
                st[0].stats['longitude'] = longitude
                st[0].stats['length'] = length
                st[0].stats['depth'] = depth
                st[0].stats['obliquity'] = obliquity
                file_code = abbreviation + '_{k}'.format(k=freq_chosen)
                
                print(file_code)
                print(st[0].stats.endtime)
                tr.plot();
                instrument = pd.DataFrame({'Network':[st[0].stats.network],'Creepmeter_abbrv':['{k}'.format(k=abbreviation)],'File_code':[file_code],
                                            'Start Time':[st[0].stats.starttime],'End Time':[st[0].stats.endtime],'Sampling rate, Hz':[st[0].stats.sampling_rate],
                                            'Sampling rate, mins':[freq_chosen],'Latitude':[latitude],'Longitude':[longitude],'Length':[length],'Depth':[depth],
                                            'Obliquity':[obliquity]})
                
                Creepmeter_dataframe_SAC = pd.concat([Creepmeter_dataframe_SAC,instrument],ignore_index=True)
                
                Creepmeter_dataframe_SAC.drop_duplicates(subset='File_code',inplace=True)
                Creepmeter_dataframe_SAC.reset_index(inplace=True,drop=True)
                
                Creepmeter_dataframe_SAC.to_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes.csv')
                st.write('../../Data/DATA_tidied/SAC/{k}'.format(k=file_code),format='SAC')
            except KeyError:
                print('key not found')
            

In [54]:
st = obspy.read('../../Data/DATA_tidied/SAC/')

In [ ]:
data.keys

In [ ]:
st[0].plot()